# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import re
import nltk
import pickle
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import multioutput
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_res.db')
df = pd.read_sql ('SELECT * FROM disaster_res', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# define the independant variable (X) and the dependant variable (Y)
X = df.message
ycol = df.columns[4:]
Y = df.iloc[:,4:]

In [4]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    stop_words = stopwords.words("english")
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# use train_test_split function to split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, Y)


In [8]:
# implement the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict (X_test)
y_pred = pd.DataFrame (y_pred, columns = y_test.columns)

In [10]:
y_pred.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2,1,1,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#model metrics 
from sklearn.metrics import classification_report
print(classification_report(y_test['related'], y_pred['related']))

             precision    recall  f1-score   support

          0       0.69      0.12      0.20      1533
          1       0.78      0.98      0.87      4972
          2       0.62      0.10      0.18        49

avg / total       0.76      0.77      0.71      6554



In [12]:
predicted_classes = Y.columns
predicted_classes

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [13]:
#run the classification report over the whole output dataframe
def metrics_class(col, y_actual, y_calc):
    return classification_report(y_actual[col], y_calc[col])

for column in predicted_classes:
    print('Metrics of teh classification pipeline on ""', column, '"" outcome is :' )
    print(metrics_class(column, y_test, y_pred))

Metrics of teh classification pipeline on "" related "" outcome is :
             precision    recall  f1-score   support

          0       0.69      0.12      0.20      1533
          1       0.78      0.98      0.87      4972
          2       0.62      0.10      0.18        49

avg / total       0.76      0.77      0.71      6554

Metrics of teh classification pipeline on "" request "" outcome is :
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      5476
          1       0.71      0.50      0.59      1078

avg / total       0.87      0.88      0.88      6554

Metrics of teh classification pipeline on "" offer "" outcome is :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      0.99      0.99      6554

Metrics of teh classification pipeline on "" aid_related "" outcome is :
             precision  

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
# get the model parameters to select from them in the grid search step 
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7efdeded0620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [15]:
# from the printed model parameters above, the selected parameters for the grid search are 
# stored in parameters_grid
parameters_grid = {'clf__estimator__learning_rate': [0.5, 1],
              'clf__estimator__n_estimators': [30, 60]}


#define the gridsearch object and run over the training data to fit the model
cv = GridSearchCV(pipeline, param_grid=parameters_grid)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [0.5, 1], 'clf__estimator__n_estimators': [30, 60]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [16]:
cv.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__vect', 'estimator__tfidf', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__preprocessor', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__vocabulary', 'estimator__tfidf__norm', 'estimator__tfidf__smooth_idf', 'estimator__tfidf__sublinear_tf', 'estimator__tfidf__use_idf', 'estimator__clf__estimator__algorithm', 'estimator__clf__estimator__base_estimator', 'estimator__clf__estimator__learning_rate', 'estimator__clf__estimator__n_estimators', 'estimator__clf__estimator__random_state', 'estimator__clf__estim

In [17]:
print(cv)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [0.5, 1], 'clf__estimator__n_estimators': [30, 60]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)


In [18]:
# the predcted labels consderng the grd search
ytest_pred_cv = cv.predict(X_test)

In [19]:
ytest_pred_cv = pd.DataFrame (y_pred, columns = predicted_classes)

In [20]:
ytest_pred_cv.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2,1,1,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
y_test.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7573,1,1,0,1,1,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
13243,1,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
12082,1,1,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
19705,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20438,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
# get the classification report 
for column in predicted_classes:
    print('Metrics of teh classification pipeline on ""', column, '"" outcome is :' )
    print(metrics_class(column, y_test, ytest_pred_cv))

Metrics of teh classification pipeline on "" related "" outcome is :
             precision    recall  f1-score   support

          0       0.69      0.12      0.20      1533
          1       0.78      0.98      0.87      4972
          2       0.62      0.10      0.18        49

avg / total       0.76      0.77      0.71      6554

Metrics of teh classification pipeline on "" request "" outcome is :
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      5476
          1       0.71      0.50      0.59      1078

avg / total       0.87      0.88      0.88      6554

Metrics of teh classification pipeline on "" offer "" outcome is :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      0.99      0.99      6554

Metrics of teh classification pipeline on "" aid_related "" outcome is :
             precision  

In [23]:
"""

def display_results(cv, y_test, y_pred):
    for cl in predicted_classes:
        labels = np.unique(y_pred[cl])
        confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
        accuracy = (y_pred[cl] == y_test[cl]).mean()
        
        print(cl)
        print("Labels:", labels)
        print("Confusion Matrix:\n", confusion_mat)
        print("Accuracy:", accuracy)
        print("\nBest Parameters:", cv.best_params_)


def main():
    model = cv
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred = pd.DataFrame (y_pred, columns = y_test.columns)

    display_results(model, y_test, y_pred)

main()

"""

'\n\ndef display_results(cv, y_test, y_pred):\n    for cl in predicted_classes:\n        labels = np.unique(y_pred[cl])\n        confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)\n        accuracy = (y_pred[cl] == y_test[cl]).mean()\n        \n        print(cl)\n        print("Labels:", labels)\n        print("Confusion Matrix:\n", confusion_mat)\n        print("Accuracy:", accuracy)\n        print("\nBest Parameters:", cv.best_params_)\n\n\ndef main():\n    model = cv\n    model.fit(X_train, y_train)\n    y_pred = model.predict(X_test)\n    y_pred = pd.DataFrame (y_pred, columns = y_test.columns)\n\n    display_results(model, y_test, y_pred)\n\nmain()\n\n'

In [24]:
cv.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__vect', 'estimator__tfidf', 'estimator__clf', 'estimator__vect__analyzer', 'estimator__vect__binary', 'estimator__vect__decode_error', 'estimator__vect__dtype', 'estimator__vect__encoding', 'estimator__vect__input', 'estimator__vect__lowercase', 'estimator__vect__max_df', 'estimator__vect__max_features', 'estimator__vect__min_df', 'estimator__vect__ngram_range', 'estimator__vect__preprocessor', 'estimator__vect__stop_words', 'estimator__vect__strip_accents', 'estimator__vect__token_pattern', 'estimator__vect__tokenizer', 'estimator__vect__vocabulary', 'estimator__tfidf__norm', 'estimator__tfidf__smooth_idf', 'estimator__tfidf__sublinear_tf', 'estimator__tfidf__use_idf', 'estimator__clf__estimator__algorithm', 'estimator__clf__estimator__base_estimator', 'estimator__clf__estimator__learning_rate', 'estimator__clf__estimator__n_estimators', 'estimator__clf__estimator__random_state', 'estimator__clf__estim

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [25]:
ext = pickle.dumps('MLPipeLine.pkl')

NameError: name 'pickle' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.